## Sistema de Recomendación


Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación de películas. El EDA debería incluir gráficas interesantes para extraer datos, como por ejemplo una nube de palabras con las palabras más frecuentes en los títulos de las películas. Éste consiste en recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas, se ordenarán según el score de similaridad y devolverá una lista de Python con 5 valores, cada uno siendo el string del nombre de las películas con mayor puntaje, en orden descendente.

def recomendacion( titulo ): Se ingresa el nombre de una película y te recomienda las similares en una lista de 5 valores.

Lo que tenemos que hacer primero es importar las librerías de scikit-learn. En este caso tenemos TfidfVectorizer que convierte una colección de documentos sin procesar en una matriz y a cosine_similarity para poder utilizar su fórmula matemática.

In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from unidecode import unidecode

In [6]:
data = pd.read_parquet('C:\\Users\\User\\OneDrive\\Escritorio\\Proyecto Final Individual I\\data.parquet')


In [12]:


def normalizar_texto(texto):
    return unidecode(str(texto).lower().strip())

def combinar_features(row):
    genres = row['genre_name'] if isinstance(row['genre_name'], str) else ''
    companies = row['company_names'] if isinstance(row['company_names'], str) else ''
    return f"{genres} {companies}".strip()

def recomendar_peliculas(titulo, data, n_recomendaciones=5):
    # Verificar columnas necesarias
    required_columns = ['title', 'genre_name', 'company_names', 'release_year', 'vote_average']
    for col in required_columns:
        if col not in data.columns:
            raise ValueError(f"La columna '{col}' no está presente en el DataFrame.")
    
    # Asegurar que las columnas numéricas sean del tipo correcto
    data['release_year'] = pd.to_numeric(data['release_year'], errors='coerce')
    data['vote_average'] = pd.to_numeric(data['vote_average'], errors='coerce')
    
    # Normalizar los títulos en el dataset
    data['title_normalized'] = data['title'].apply(normalizar_texto)
    
    # Preprocesamiento
    data['combined_features'] = data.apply(combinar_features, axis=1)
    
    # Verificar si hay contenido en combined_features
    if data['combined_features'].str.strip().str.len().sum() == 0:
        print("")
        print("")
        data['combined_features'] = data['title_normalized']
    
    # Similitud del coseno
    tfidf = TfidfVectorizer(stop_words='english', min_df=1, max_df=0.9)
    tfidf_matrix = tfidf.fit_transform(data['combined_features'])
    
    # Verificar si se generó algún término
    if tfidf_matrix.shape[1] == 0:
        print("No se pudieron extraer características significativas de los datos.")
        return []
    
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Normalizar el título de búsqueda
    titulo_normalizado = normalizar_texto(titulo)
    
    # Obtener índice de la película
    idx = data.index[data['title_normalized'] == titulo_normalizado].tolist()
    if not idx:
        print(f"La película '{titulo}' no se encuentra en la base de datos.")
        return []
    idx = idx[0]
    
    # Calcular puntuaciones de similitud
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n_recomendaciones+1]
    
    # Obtener índices de películas recomendadas por similitud del coseno
    movie_indices = [i[0] for i in sim_scores]
    
    # Si no hay suficientes recomendaciones, usar KNN
    if len(movie_indices) < n_recomendaciones:
        # Preparar características para KNN
        genres = pd.get_dummies(data['genre_name'].fillna('').str.split().apply(pd.Series).stack()).groupby(level=0).sum()
        companies = pd.get_dummies(data['company_names'].fillna('').str.split().apply(pd.Series).stack()).groupby(level=0).sum()
        
        # Normalizar el año y el puntaje
        scaler = MinMaxScaler()
        years = scaler.fit_transform(data['release_year'].values.reshape(-1, 1))
        ratings = scaler.fit_transform(data['vote_average'].fillna(data['vote_average'].mean()).values.reshape(-1, 1))
        
        # Combinar características
        features = np.hstack((genres.values, companies.values, years, ratings))
        
        # Entrenar KNN
        knn = NearestNeighbors(n_neighbors=n_recomendaciones, metric='euclidean')
        knn.fit(features)
        
        # Encontrar vecinos más cercanos
        _, indices = knn.kneighbors(features[idx].reshape(1, -1))
        
        # Añadir índices faltantes
        movie_indices.extend([i for i in indices[0] if i not in movie_indices])
        movie_indices = movie_indices[:n_recomendaciones]
    
    # Devolver las películas recomendadas con sus puntajes
    recomendaciones = data[['title', 'vote_average']].iloc[movie_indices]
    return recomendaciones.values.tolist()

# Ejemplo de uso con input del usuario
titulo_pelicula = input("Dime una pelícua y te recomendaré 5 películas")
recomendaciones = recomendar_peliculas(titulo_pelicula, data)
if recomendaciones:
    print(f"\nRecomendaciones para '{titulo_pelicula}':")
    for i, (pelicula, puntaje) in enumerate(recomendaciones, 1):
        print(f"{i}. {pelicula} (Puntaje: {puntaje:.1f})")
else:
    print("No se pudieron generar recomendaciones.")




Recomendaciones para 'jumanji':
1. Toy Story (Puntaje: 7.7)
2. Grumpier Old Men (Puntaje: 6.5)
3. Waiting to Exhale (Puntaje: 6.1)
4. Father of the Bride Part II (Puntaje: 5.7)
5. Heat (Puntaje: 7.7)
